**Computes large population movements**

**Input**: csv using the following columns:
* `transportation_mode_tr`: used mode of transport for the trip, in readable format
* `longitude` and `latitude`: Starting point of trip
* `end_longitude` and `end_latitude`: Ending point of trip
* `user_id`: Id of the traveling user, used to make sure results include more than 3 users per geographic division

**Output**: geoJSON file "../static/data/exode.geojson" containing h3 cell shapes with the following metadata:
* `geometry`: h3 shape of the destinations
* `MostCommonTransport`: prefered `transportation_mode_tr` for most trips
* `color`: A color representation of `MostCommonTransport`
* `Count`: Total number of trips arriving in cell

**Output**: geoJSON file "../static/data/exode_lines.geojson" containing line shapes with the following metadata:
* `geometry`: LineString strating from paris and ending in the centroid of h3 celles
* `MostCommonTransport`: prefered `transportation_mode_tr` for most trips
* `color`: A color representation of `MostCommonTransport`
* `Count`: Total number of trips arriving in cell

TODO: replicate outputs for incoming trips

In [ ]:
import pandas as pd
from mappymatch.constructs.geofence import Geofence
from shapely.geometry import Point, LineString
from shapely.vectorized import contains
import h3pandas
import geopandas as gpd
import folium
import json
import numpy as np


In [ ]:
geofence_idf = Geofence.from_geojson("sources/region-ile-de-france.geojson")

In [ ]:
df = pd.read_csv("sources/data_min.csv")

In [ ]:
df

In [ ]:
geofence_paris_geometry = geofence_paris.geometry

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")


In [ ]:
geofence_idf_geometry = geofence_idf.geometry

# Filter points within the geofence
gdf_from_idf = gdf[gdf.geometry.within(geofence_idf_geometry)]

# Create end_geometry for the end points
gdf_from_idf['end_geometry'] = gpd.points_from_xy(gdf_from_idf.end_longitude, gdf_from_idf.end_latitude)

# Filter points where end location is outside the geofence
gdf_exit_idf = gdf_from_idf[~gdf_from_idf['end_geometry'].within(geofence_idf_geometry)]

# Drop the temporary 'end_geometry' column if not needed
gdf_exit_idf = gdf_exit_idf.drop(columns=['geometry', 'end_geometry'])
gdf_exit_idf

In [ ]:
dfh3 = gdf_exit_idf.h3.geo_to_h3(4, lat_col="end_latitude", lng_col="end_longitude", set_index=False)
df_unique_user = dfh3.drop_duplicates(subset=['h3_04', 'user_id'])
drawgeoframe = df_unique_user[['h3_04']].groupby(['h3_04']).agg(Count=('h3_04', np.size))
drawgeoframe=drawgeoframe.reset_index().set_index('h3_04')
drawgeoframe = drawgeoframe[drawgeoframe['Count'] > 3]
drawgeoframe = drawgeoframe.h3.h3_to_geo()
drawgeoframe["center_geom"] = drawgeoframe["geometry"]
drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()
drawgeoframe

In [ ]:
fixed_point = Point(2.333333, 48.866667)

# Function to create a line from the fixed point to each point
def create_line(point):
    return LineString([fixed_point, point])

# Apply the function to each geometry in the GeoDataFrame
drawgeoframe['geom'] = drawgeoframe['center_geom'].apply(create_line)
drawgeoframe.max()

In [ ]:
import branca.colormap as cm
colormap = cm.LinearColormap(["green", "yellow", "red"], vmin=0, vmax=50)
drawgeoframe["color"] = drawgeoframe["Count"].apply(lambda x: colormap(x)[:-2])

start_lat = 48.8915079
start_long = 2.3495425
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.GeoJson(drawgeoframe[["geometry", "color"]], style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
folium.GeoJson(drawgeoframe[["geom", "color"]].rename(columns={"geom": "geometry"}), style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
m

In [ ]:
def most_common_value(series):
    return series.mode().iloc[0]
dfh3 = gdf_exit_idf.h3.geo_to_h3(4, lat_col="end_latitude", lng_col="end_longitude", set_index=False)
df_unique_user = dfh3.drop_duplicates(subset=['h3_04', 'user_id'])
drawgeoframe = df_unique_user[['h3_04', 'transportation_mode_tr']].groupby(['h3_04']).agg(Count=('h3_04', np.size), MostCommonTransport=('transportation_mode_tr', most_common_value))
drawgeoframe=drawgeoframe.reset_index().set_index('h3_04')
drawgeoframe = drawgeoframe[drawgeoframe['Count'] > 3]
drawgeoframe = drawgeoframe.h3.h3_to_geo()
drawgeoframe["center_geom"] = drawgeoframe["geometry"]
drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()
drawgeoframe

In [ ]:
fixed_point = Point(2.333333, 48.866667)

# Function to create a line from the fixed point to each point
def create_line(point):
    return LineString([fixed_point, point])

# Apply the function to each geometry in the GeoDataFrame
drawgeoframe['geom'] = drawgeoframe['center_geom'].apply(create_line)

In [ ]:
colormap = {
    "PLANE": "red",
    "PASSENGER_CAR": "orange",
    "RAIL_TRIP": "green",
    "HIGH_SPEED_TRAIN": "green",
    "FOOT": "black" # wtf
}
drawgeoframe["color"] = drawgeoframe["MostCommonTransport"].apply(lambda x: colormap.get(x))

start_lat = 48.8915079
start_long = 2.3495425
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.GeoJson(drawgeoframe[["geometry", "color"]], style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
drawgeoframe[["geometry", "color", "Count", "MostCommonTransport"]].to_file("../static/data/exode.geojson", driver="GeoJSON")
folium.GeoJson(drawgeoframe[["geom", "color"]].rename(columns={"geom": "geometry"}), style_function=lambda f: {"weight": "0.5", "color": f['properties']['color']}).add_to(m)
drawgeoframe[["geom", "color", "Count", "MostCommonTransport"]].rename(columns={"geom": "geometry"}).to_file("../static/data/exode_lines.geojson", driver="GeoJSON")
m

In [ ]:
geofence_idf_geometry = geofence_idf.geometry

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")

# Filter points outside the geofence
gdf_from_idf = gdf[~gdf.geometry.within(geofence_idf_geometry)]

# Create end_geometry for the end points
gdf_from_idf['end_geometry'] = gpd.points_from_xy(gdf_from_idf.end_longitude, gdf_from_idf.end_latitude)

# Filter points where end location is inside the geofence
gdf_enter_idf = gdf_from_idf[gdf_from_idf['end_geometry'].within(geofence_idf_geometry)]

# Drop the temporary 'end_geometry' column if not needed
gdf_enter_idf = gdf_enter_idf.drop(columns='end_geometry')
gdf_enter_idf


In [ ]:
dfh3 = gdf_enter_idf.h3.geo_to_h3(6, lat_col="latitude", lng_col="longitude", set_index=False)
df_unique_user = dfh3.drop_duplicates(subset=['h3_06', 'user_id'])
drawgeoframe = df_unique_user[['h3_06']].groupby(['h3_06']).agg(Count=('h3_06', np.size))
drawgeoframe=drawgeoframe.reset_index().set_index('h3_06')
drawgeoframe = drawgeoframe[drawgeoframe['Count'] > 3]
drawgeoframe = drawgeoframe.h3.h3_to_geo()
drawgeoframe["center_geom"] = drawgeoframe["geometry"]
drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()
drawgeoframe